In [ ]:
!pip install pytorch-lightning sentence-transformers torchmetrics rich

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
import os

os.chdir('/content/drive/MyDrive/NLP Group Project/synthetic-squad-main/src')
os.getcwd()

'/content/drive/MyDrive/NLP Group Project/synthetic-squad-main/src'

# Roberta Baseline

## Task 3

### Prepare the Data

In [8]:
import pandas as pd
from datautils import SynDataModule, SynBatcher

TASK = 3
EDPATH = r'/content/drive/MyDrive/NLP Group Project/Texts/punc_data_tr.csv'
MPATH = "roberta-base"

df = pd.read_csv(EDPATH)

label2id = {label: idx for idx, label in enumerate(df['alg'].unique())}

In [ ]:
batcher = SynBatcher(tnkzr_path=MPATH)
train_dm = SynDataModule(data_path=EDPATH, task='aa', label2id=label2id, batcher=batcher)
# train_dm.setup('fit')

### Prepare the model

In [ ]:
from modelling import SequenceClassification
from pytorch_lightning.callbacks import(
    EarlyStopping,
    ModelCheckpoint,
    RichModelSummary
)

SAVE_PATH = "/content/drive/MyDrive/NLP Group Project/synthetic-squad-main/models/task3/roberta-base"

model = SequenceClassification(MPATH, num_labels=len(label2id))
# print("Loaded fresh model")

# model = SequenceClassification.load_from_checkpoint("/content/drive/MyDrive/NLP Group Project/synthetic-squad-main/models/task3/roberta-base/task-3-epoch=2-val_loss=0.20.ckpt")

checkpoint_callback = ModelCheckpoint(
    SAVE_PATH,
    filename=f'task-{TASK}-{{epoch}}-{{val_loss:.2f}}',
    monitor='val_loss',
    save_weights_only=True,
)
early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=1e-4, patience=8,
    verbose=False,
    mode="min"
)

### Train the model

In [ ]:
from pytorch_lightning import Trainer, seed_everything


seed_everything(42, workers=True)

trainer = Trainer(
    max_epochs=-1,
    deterministic=True,
    accumulate_grad_batches=4,
    callbacks=[checkpoint_callback, early_stop_callback, RichModelSummary()],
    accelerator='gpu',
    log_every_n_steps=16
)

trainer.fit(model, train_dm)

## Task 2

### Prepare the Data

In [5]:
TASK = 2

label2id = {
    'human': 0,
    'machine': 1 
}

In [9]:
batcher = SynBatcher(tnkzr_path=MPATH)
train_dm = SynDataModule(data_path=EDPATH, task='hvm', label2id=label2id, batcher=batcher)
# train_dm.setup('fit')

OSError: ignored

### Prepare the model

In [ ]:
SAVE_PATH = "../models/task2/roberta-base-cased"

model = SequenceClassification(MPATH, num_labels=len(label2id))
# print("Loaded fresh model")
#model = SequenceClassification.load_from_checkpoint("../models/task2/roberta-base-cased/task-2-epoch=1-val_loss=0.03.ckpt")

checkpoint_callback = ModelCheckpoint(
    SAVE_PATH,
    filename=f'task-{TASK}-{{epoch}}-{{val_loss:.2f}}',
    monitor='val_loss',
    save_weights_only=True,
)
early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=1e-4, patience=8,
    verbose=False,
    mode="min"
)

### Train the model

In [ ]:
trainer = Trainer(
    max_epochs=-1,
    deterministic=True,
    accumulate_grad_batches=4,
    callbacks=[checkpoint_callback, early_stop_callback, RichModelSummary()],
    accelerator='gpu',
    log_every_n_steps=16
)

# trainer.fit(model, train_dm)

In [ ]:
trainer.test(model, eval_dm)